# Filesystem Tool

The `uwtools` API's `fs` module provides functions to copy and link files as well as create directories. 

<div class="alert alert-warning"><b>Note: </b>This notebook was tested using <code>uwtools</code> version 2.6.0. </div>
<div class="alert alert-info">For more information, please see the <a href="https://uwtools.readthedocs.io/en/2.5.0/sections/user_guide/api/fs.html">uwtools.api.fs</a> Read the Docs page.</div>

## Table of Contents

* [Copying Files](#Copying-Files)
    * [Failing to Copy](#Failing-to-Copy)
    * [Using the `key_path` Argument](#Using-the-key_path-Argument)
    * [Using the `Copier` Class](#Using-the-Copier-Class)
* [Linking Files](#Linking-Files)
    * [Failing to Link](#Failing-to-Link)
    * [Using the `key_path` Argument](#Using-the-key_path-Argument-)
    * [Using the `Linker` Class](#Using-the-Linker-Class)
* [Creating Directories](#Creating-Directories)
    * [Using the `key_path` Argument](#Using-the-key_path-Argument--)
    * [Using the `MakeDirs` Class](#Using-the-MakeDirs-Class)
* [Using glob Patterms](#Using-glob-Patterns)
* [Copying HTTP Sources](#Copying-HTTP-Sources)

In [1]:
from pathlib import Path
from shutil import rmtree
from uwtools.api import fs
from uwtools.api.logging import use_uwtools_logger
import yaml

use_uwtools_logger()

## Copying Files

The `copy()` function copies files, automatically creating parent directories as needed.

In [2]:
help(fs.copy)

Help on function copy in module uwtools.api.fs:

copy(config: Union[dict, str, pathlib.Path, NoneType] = None, target_dir: Union[str, pathlib.Path, NoneType] = None, cycle: Optional[datetime.datetime] = None, leadtime: Optional[datetime.timedelta] = None, key_path: Optional[list[Union[bool, float, int, str]]] = None, dry_run: bool = False, stdin_ok: bool = False) -> dict[str, list[str]]
    Copy files.
    
    :param config: YAML-file path, or ``dict`` (read ``stdin`` if missing or ``None``).
    :param target_dir: Path to target directory.
    :param cycle: A datetime object to make available for use in the config.
    :param leadtime: A timedelta object to make available for use in the config.
    :param key_path: Path of keys to config block to use.
    :param dry_run: Do not copy files.
    :param stdin_ok: OK to read from ``stdin``?
    :return: A report on files copied / not copied.



Files to be copied are specified by a mapping from keys destination-pathname keys to source-pathname values, either in a YAML file or a a Python ``dict``.

In [3]:
%%bash
cat fixtures/fs/config/copy.yaml

file1-copy.nml: fixtures/fs/data/file1.nml
data/file2-copy.txt: fixtures/fs/data/file2.txt
data/file3-copy.csv: fixtures/fs/data/file3.csv


With these instructions, `copy()` creates a copy of each given file with the given name and in the given subdirectory. Copies are created in the directory indicated by `target_dir`. Paths can be provided either as a string or <a href="https://docs.python.org/3/library/pathlib.html#pathlib.Path">Path</a> object. Any directories in the targeted paths for copying will be created if they don't already exist. The return value of `copy()` is a `dict` reporting files that were created (`ready`) and not created (`not-ready`).

In [4]:
rmtree("tmp/copy-target", ignore_errors=True)
fs.copy(
    config="fixtures/fs/config/copy.yaml",
    target_dir=Path("tmp/copy-target")
)

[2025-02-22T23:14:02]     INFO Validating config against internal schema: files-to-stage
[2025-02-22T23:14:02]     INFO 0 schema-validation errors found in fs config
[2025-02-22T23:14:02]     INFO File fixtures/fs/data/file1.nml: Ready
[2025-02-22T23:14:02]     INFO File fixtures/fs/data/file2.txt: Ready
[2025-02-22T23:14:02]     INFO File fixtures/fs/data/file3.csv: Ready
[2025-02-22T23:14:02]     INFO Copy fixtures/fs/data/file1.nml -> tmp/copy-target/file1-copy.nml: Executing
[2025-02-22T23:14:02]     INFO Copy fixtures/fs/data/file1.nml -> tmp/copy-target/file1-copy.nml: Ready
[2025-02-22T23:14:02]     INFO Copy fixtures/fs/data/file2.txt -> tmp/copy-target/data/file2-copy.txt: Executing
[2025-02-22T23:14:02]     INFO Copy fixtures/fs/data/file2.txt -> tmp/copy-target/data/file2-copy.txt: Ready
[2025-02-22T23:14:02]     INFO Copy fixtures/fs/data/file3.csv -> tmp/copy-target/data/file3-copy.csv: Executing
[2025-02-22T23:14:02]     INFO Copy fixtures/fs/data/file3.csv -> tmp/copy-ta

{'ready': ['tmp/copy-target/file1-copy.nml',
  'tmp/copy-target/data/file2-copy.txt',
  'tmp/copy-target/data/file3-copy.csv'],
 'not-ready': []}

Examining the target directory, we can see that the copies of the files have been made with their specified names and in their specified directories.

In [5]:
%%bash
tree tmp/copy-target

tmp/copy-target
├── data
│   ├── file2-copy.txt
│   └── file3-copy.csv
└── file1-copy.nml

2 directories, 3 files


### Failing to Copy

A configuration can be provided as a dictionary instead as this example demonstrates. However, `missing-file.nml` does not exist. The function provides a warning and returns an accurate run report.

In [6]:
fs.copy(
    config={"missing-copy.nml": "fixtures/fs/data/missing-file.nml"},
    target_dir="tmp/copy-target"
)

[2025-02-22T23:14:02]     INFO Validating config against internal schema: files-to-stage
[2025-02-22T23:14:02]     INFO 0 schema-validation errors found in fs config
[2025-02-22T23:14:02]  WARNING File fixtures/fs/data/missing-file.nml: Not ready [external asset]
[2025-02-22T23:14:02]  WARNING Copy fixtures/fs/data/missing-file.nml -> tmp/copy-target/missing-copy.nml: Not ready
[2025-02-22T23:14:02]  WARNING Copy fixtures/fs/data/missing-file.nml -> tmp/copy-target/missing-copy.nml: Requires:
[2025-02-22T23:14:02]  WARNING Copy fixtures/fs/data/missing-file.nml -> tmp/copy-target/missing-copy.nml: ✖ File fixtures/fs/data/missing-file.nml
[2025-02-22T23:14:02]  WARNING File copies: Not ready
[2025-02-22T23:14:02]  WARNING File copies: Requires:
[2025-02-22T23:14:02]  WARNING File copies: ✖ Copy fixtures/fs/data/missing-file.nml -> tmp/copy-target/missing-copy.nml


{'ready': [], 'not-ready': ['tmp/copy-target/missing-copy.nml']}

The missing copy does not appear in the target directory.

In [7]:
%%bash
tree tmp/copy-target

tmp/copy-target
├── data
│   ├── file2-copy.txt
│   └── file3-copy.csv
└── file1-copy.nml

2 directories, 3 files


### Using the `key_path` Argument<!--copy-->

Consider the following configuration, in which the destination/source mapping is not located at the top level of the configuration:

In [8]:
%%bash
cat fixtures/fs/config/copy-keys.yaml

files:
  to:
    copy:
      file1-copy.nml: fixtures/fs/data/file1.nml
      data/file2-copy.txt: fixtures/fs/data/file2.txt
      data/file3-copy.csv: fixtures/fs/data/file3.csv


Without additional information, `copy()` would raise a `UWConfigError` given this configuration. However, the list of keys leading to the destination/source mapping can be provided with the `key_path` argument:

In [9]:
rmtree("tmp/copy-keys-target", ignore_errors=True)
fs.copy(
    config="fixtures/fs/config/copy-keys.yaml",
    target_dir="tmp/copy-keys-target",
    key_path=["files","to","copy"]
)

[2025-02-22T23:14:02]     INFO Validating config against internal schema: files-to-stage
[2025-02-22T23:14:02]     INFO 0 schema-validation errors found in fs config
[2025-02-22T23:14:02]     INFO File fixtures/fs/data/file1.nml: Ready
[2025-02-22T23:14:02]     INFO File fixtures/fs/data/file2.txt: Ready
[2025-02-22T23:14:02]     INFO File fixtures/fs/data/file3.csv: Ready
[2025-02-22T23:14:02]     INFO Copy fixtures/fs/data/file1.nml -> tmp/copy-keys-target/file1-copy.nml: Executing
[2025-02-22T23:14:02]     INFO Copy fixtures/fs/data/file1.nml -> tmp/copy-keys-target/file1-copy.nml: Ready
[2025-02-22T23:14:02]     INFO Copy fixtures/fs/data/file2.txt -> tmp/copy-keys-target/data/file2-copy.txt: Executing
[2025-02-22T23:14:02]     INFO Copy fixtures/fs/data/file2.txt -> tmp/copy-keys-target/data/file2-copy.txt: Ready
[2025-02-22T23:14:02]     INFO Copy fixtures/fs/data/file3.csv -> tmp/copy-keys-target/data/file3-copy.csv: Executing
[2025-02-22T23:14:02]     INFO Copy fixtures/fs/data

{'ready': ['tmp/copy-keys-target/file1-copy.nml',
  'tmp/copy-keys-target/data/file2-copy.txt',
  'tmp/copy-keys-target/data/file3-copy.csv'],
 'not-ready': []}

With this information provided, the copy is successful.

In [10]:
%%bash
tree tmp/copy-keys-target

tmp/copy-keys-target
├── data
│   ├── file2-copy.txt
│   └── file3-copy.csv
└── file1-copy.nml

2 directories, 3 files


### Using the `Copier` Class

An alternative to using `copy()` is to instantiate a `Copier` object , then call its `go()` method.

In [11]:
help(fs.Copier)

Help on class Copier in module uwtools.fs:

class Copier(FileStager)
 |  Copier(config: Union[dict, str, pathlib.Path, NoneType] = None, target_dir: Union[str, pathlib.Path, NoneType] = None, cycle: Optional[datetime.datetime] = None, leadtime: Optional[datetime.timedelta] = None, key_path: Optional[list[Union[bool, float, int, str]]] = None) -> None
 |  
 |  Stage files by copying.
 |  
 |  Method resolution order:
 |      Copier
 |      FileStager
 |      Stager
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  go(self)
 |      Copy files.
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __abstractmethods__ = frozenset()
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Stager:
 |  
 |  __init__(self, config: Union[dict, str, pathlib.Path, NoneType] = None, target_dir: Union[str, pathlib.Path, NoneType] = None, cycl

A `Copier` object is instantiated using the same parameters as `copy()`, but copying is not performed until `Copier.go()` is called.

In [12]:
rmtree("tmp/copier-target", ignore_errors=True)
copier = fs.Copier(
    config="fixtures/fs/config/copy.yaml",
    target_dir="tmp/copier-target"
)
copier.go()

[2025-02-22T23:14:02]     INFO Validating config against internal schema: files-to-stage
[2025-02-22T23:14:02]     INFO 0 schema-validation errors found in fs config
[2025-02-22T23:14:02]     INFO File fixtures/fs/data/file1.nml: Ready
[2025-02-22T23:14:02]     INFO File fixtures/fs/data/file2.txt: Ready
[2025-02-22T23:14:02]     INFO File fixtures/fs/data/file3.csv: Ready
[2025-02-22T23:14:02]     INFO Copy fixtures/fs/data/file1.nml -> tmp/copier-target/file1-copy.nml: Executing
[2025-02-22T23:14:02]     INFO Copy fixtures/fs/data/file1.nml -> tmp/copier-target/file1-copy.nml: Ready
[2025-02-22T23:14:02]     INFO Copy fixtures/fs/data/file2.txt -> tmp/copier-target/data/file2-copy.txt: Executing
[2025-02-22T23:14:02]     INFO Copy fixtures/fs/data/file2.txt -> tmp/copier-target/data/file2-copy.txt: Ready
[2025-02-22T23:14:02]     INFO Copy fixtures/fs/data/file3.csv -> tmp/copier-target/data/file3-copy.csv: Executing
[2025-02-22T23:14:02]     INFO Copy fixtures/fs/data/file3.csv -> t

File copies <281472708104976>

Once `Copier.go()` is called, copies are created in the same way as they would have with `copy()`.

In [13]:
%%bash
tree tmp/copier-target

tmp/copier-target
├── data
│   ├── file2-copy.txt
│   └── file3-copy.csv
└── file1-copy.nml

2 directories, 3 files


## Linking Files

The `link()` function creates symbolic links to files, automatically creating parent directories as needed.

In [14]:
help(fs.link)

Help on function link in module uwtools.api.fs:

link(config: Union[dict, str, pathlib.Path, NoneType] = None, target_dir: Union[str, pathlib.Path, NoneType] = None, cycle: Optional[datetime.datetime] = None, leadtime: Optional[datetime.timedelta] = None, key_path: Optional[list[Union[bool, float, int, str]]] = None, dry_run: bool = False, stdin_ok: bool = False) -> dict[str, list[str]]
    Link files.
    
    :param config: YAML-file path, or ``dict`` (read ``stdin`` if missing or ``None``).
    :param target_dir: Path to target directory.
    :param cycle: A datetime object to make available for use in the config.
    :param leadtime: A timedelta object to make available for use in the config.
    :param key_path: Path of keys to config block to use.
    :param dry_run: Do not link files.
    :param stdin_ok: OK to read from ``stdin``?
    :return: A report on files linked / not linked.



Links to be created are specified by a mapping from keys destination-pathname keys to source-pathname values, either in a YAML file or a Python ``dict``.

In [15]:
%%bash
cat fixtures/fs/config/link.yaml

file1-link.nml: fixtures/fs/data/file1.nml
file2-link.txt: fixtures/fs/data/file2.txt
data/file3-link.csv: fixtures/fs/data/file3.csv


With these instructions, `link()` creates a symbolic link of each given file with the given name and in the given subdirectory. Links are created in the directory indicated by `target_dir`. Paths can be provided either as a string or <a href="https://docs.python.org/3/library/pathlib.html#pathlib.Path">Path</a> object. Any directories in the targeted paths will be created if they don't already exist. The return value of `link()` is a `dict` reporting files that were created (`ready`) and not created (`not-ready`).

In [16]:
rmtree("tmp/link-target", ignore_errors=True)
fs.link(
    config=Path("fixtures/fs/config/link.yaml"),
    target_dir="tmp/link-target"
)

[2025-02-22T23:14:02]     INFO Validating config against internal schema: files-to-stage
[2025-02-22T23:14:02]     INFO 0 schema-validation errors found in fs config
[2025-02-22T23:14:03]     INFO Filesystem item fixtures/fs/data/file1.nml: Ready
[2025-02-22T23:14:03]     INFO Filesystem item fixtures/fs/data/file2.txt: Ready
[2025-02-22T23:14:03]     INFO Filesystem item fixtures/fs/data/file3.csv: Ready
[2025-02-22T23:14:03]     INFO Link tmp/link-target/file1-link.nml -> fixtures/fs/data/file1.nml: Executing
[2025-02-22T23:14:03]     INFO Link tmp/link-target/file1-link.nml -> fixtures/fs/data/file1.nml: Ready
[2025-02-22T23:14:03]     INFO Link tmp/link-target/file2-link.txt -> fixtures/fs/data/file2.txt: Executing
[2025-02-22T23:14:03]     INFO Link tmp/link-target/file2-link.txt -> fixtures/fs/data/file2.txt: Ready
[2025-02-22T23:14:03]     INFO Link tmp/link-target/data/file3-link.csv -> fixtures/fs/data/file3.csv: Executing
[2025-02-22T23:14:03]     INFO Link tmp/link-target/da

{'ready': ['tmp/link-target/file1-link.nml',
  'tmp/link-target/file2-link.txt',
  'tmp/link-target/data/file3-link.csv'],
 'not-ready': []}

Examining the target directory, we can see that the links have been created with their specified names and in their specified directories.

In [17]:
%%bash
tree tmp/link-target

tmp/link-target
├── data
│   └── file3-link.csv -> ../../../fixtures/fs/data/file3.csv
├── file1-link.nml -> ../../fixtures/fs/data/file1.nml
└── file2-link.txt -> ../../fixtures/fs/data/file2.txt

2 directories, 3 files


### Failing to Link

A configuration can be provided as a dictionary instead as this example demonstrates. However, `missing-file.nml` does not exist. The function provides a warning and returns an accurate run report.

In [18]:
fs.link(
    config={"missing-link.nml":"fixtures/fs/missing-file.nml"},
    target_dir="tmp/link-target"
)

[2025-02-22T23:14:03]     INFO Validating config against internal schema: files-to-stage
[2025-02-22T23:14:03]     INFO 0 schema-validation errors found in fs config
[2025-02-22T23:14:03]  WARNING Filesystem item fixtures/fs/missing-file.nml: Not ready [external asset]
[2025-02-22T23:14:03]  WARNING Link tmp/link-target/missing-link.nml -> fixtures/fs/missing-file.nml: Not ready
[2025-02-22T23:14:03]  WARNING Link tmp/link-target/missing-link.nml -> fixtures/fs/missing-file.nml: Requires:
[2025-02-22T23:14:03]  WARNING Link tmp/link-target/missing-link.nml -> fixtures/fs/missing-file.nml: ✖ Filesystem item fixtures/fs/missing-file.nml
[2025-02-22T23:14:03]  WARNING File links: Not ready
[2025-02-22T23:14:03]  WARNING File links: Requires:
[2025-02-22T23:14:03]  WARNING File links: ✖ Link tmp/link-target/missing-link.nml -> fixtures/fs/missing-file.nml


{'ready': [], 'not-ready': ['tmp/link-target/missing-link.nml']}

The missing link does not appear in the target directory.

In [19]:
%%bash
tree tmp/link-target

tmp/link-target
├── data
│   └── file3-link.csv -> ../../../fixtures/fs/data/file3.csv
├── file1-link.nml -> ../../fixtures/fs/data/file1.nml
└── file2-link.txt -> ../../fixtures/fs/data/file2.txt

2 directories, 3 files


### Using the `key_path` Argument <!--link-->

Consider the following configuration, in which the destination/source mapping is not located at the top level of the configuration:

In [20]:
%%bash
cat fixtures/fs/config/link-keys.yaml

files:
  to:
    link:
      file1-link.nml: fixtures/fs/data/file1.nml
      file2-link.txt: fixtures/fs/data/file2.txt
      data/file3-link.csv: fixtures/fs/data/file3.csv


Without additional information, `link()` would raise a `UWConfigError` given this configuration. However, the list of keys leading to the destination/source mapping can be provided with the `Key_path` argument:

In [21]:
rmtree("tmp/link-keys-target", ignore_errors=True)
fs.link(
    config="fixtures/fs/config/link-keys.yaml",
    target_dir="tmp/link-keys-target",
    key_path=["files","to","link"]
)

[2025-02-22T23:14:03]     INFO Validating config against internal schema: files-to-stage
[2025-02-22T23:14:03]     INFO 0 schema-validation errors found in fs config
[2025-02-22T23:14:03]     INFO Filesystem item fixtures/fs/data/file1.nml: Ready
[2025-02-22T23:14:03]     INFO Filesystem item fixtures/fs/data/file2.txt: Ready
[2025-02-22T23:14:03]     INFO Filesystem item fixtures/fs/data/file3.csv: Ready
[2025-02-22T23:14:03]     INFO Link tmp/link-keys-target/file1-link.nml -> fixtures/fs/data/file1.nml: Executing
[2025-02-22T23:14:03]     INFO Link tmp/link-keys-target/file1-link.nml -> fixtures/fs/data/file1.nml: Ready
[2025-02-22T23:14:03]     INFO Link tmp/link-keys-target/file2-link.txt -> fixtures/fs/data/file2.txt: Executing
[2025-02-22T23:14:03]     INFO Link tmp/link-keys-target/file2-link.txt -> fixtures/fs/data/file2.txt: Ready
[2025-02-22T23:14:03]     INFO Link tmp/link-keys-target/data/file3-link.csv -> fixtures/fs/data/file3.csv: Executing
[2025-02-22T23:14:03]     INF

{'ready': ['tmp/link-keys-target/file1-link.nml',
  'tmp/link-keys-target/file2-link.txt',
  'tmp/link-keys-target/data/file3-link.csv'],
 'not-ready': []}

With this information provided, the links are successfully created.

In [22]:
%%bash
tree tmp/link-keys-target

tmp/link-keys-target
├── data
│   └── file3-link.csv -> ../../../fixtures/fs/data/file3.csv
├── file1-link.nml -> ../../fixtures/fs/data/file1.nml
└── file2-link.txt -> ../../fixtures/fs/data/file2.txt

2 directories, 3 files


### Using the `Linker` Class

An alternative to using `link()` is to instantiate a `Linker` object , then call its `go()` method.

In [23]:
help(fs.Linker)

Help on class Linker in module uwtools.fs:

class Linker(FileStager)
 |  Linker(config: Union[dict, str, pathlib.Path, NoneType] = None, target_dir: Union[str, pathlib.Path, NoneType] = None, cycle: Optional[datetime.datetime] = None, leadtime: Optional[datetime.timedelta] = None, key_path: Optional[list[Union[bool, float, int, str]]] = None) -> None
 |  
 |  Stage files by linking.
 |  
 |  Method resolution order:
 |      Linker
 |      FileStager
 |      Stager
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  go(self)
 |      Link files.
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __abstractmethods__ = frozenset()
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Stager:
 |  
 |  __init__(self, config: Union[dict, str, pathlib.Path, NoneType] = None, target_dir: Union[str, pathlib.Path, NoneType] = None, cycl

A `Linker` object is instantiated using the same parameters as `link()`, but links are not created until `Linker.go()` is called.

In [24]:
rmtree("tmp/linker-target", ignore_errors=True)
linker = fs.Linker(
    config="fixtures/fs/config/link.yaml",
    target_dir="tmp/linker-target"
)
linker.go()

[2025-02-22T23:14:03]     INFO Validating config against internal schema: files-to-stage
[2025-02-22T23:14:03]     INFO 0 schema-validation errors found in fs config
[2025-02-22T23:14:03]     INFO Filesystem item fixtures/fs/data/file1.nml: Ready
[2025-02-22T23:14:03]     INFO Filesystem item fixtures/fs/data/file2.txt: Ready
[2025-02-22T23:14:03]     INFO Filesystem item fixtures/fs/data/file3.csv: Ready
[2025-02-22T23:14:03]     INFO Link tmp/linker-target/file1-link.nml -> fixtures/fs/data/file1.nml: Executing
[2025-02-22T23:14:03]     INFO Link tmp/linker-target/file1-link.nml -> fixtures/fs/data/file1.nml: Ready
[2025-02-22T23:14:03]     INFO Link tmp/linker-target/file2-link.txt -> fixtures/fs/data/file2.txt: Executing
[2025-02-22T23:14:03]     INFO Link tmp/linker-target/file2-link.txt -> fixtures/fs/data/file2.txt: Ready
[2025-02-22T23:14:03]     INFO Link tmp/linker-target/data/file3-link.csv -> fixtures/fs/data/file3.csv: Executing
[2025-02-22T23:14:03]     INFO Link tmp/link

File links <281472332268880>

Once `Linker.go()` is called, links are created in the same way as they would have with `link()`.

In [25]:
%%bash
tree tmp/linker-target

tmp/linker-target
├── data
│   └── file3-link.csv -> ../../../fixtures/fs/data/file3.csv
├── file1-link.nml -> ../../fixtures/fs/data/file1.nml
└── file2-link.txt -> ../../fixtures/fs/data/file2.txt

2 directories, 3 files


## Creating Directories

The `makedirs()` function creates directories.

In [26]:
help(fs.makedirs)

Help on function makedirs in module uwtools.api.fs:

makedirs(config: Union[dict, str, pathlib.Path, NoneType] = None, target_dir: Union[str, pathlib.Path, NoneType] = None, cycle: Optional[datetime.datetime] = None, leadtime: Optional[datetime.timedelta] = None, key_path: Optional[list[Union[bool, float, int, str]]] = None, dry_run: bool = False, stdin_ok: bool = False) -> dict[str, list[str]]
    Make directories.
    
    :param config: YAML-file path, or ``dict`` (read ``stdin`` if missing or ``None``).
    :param target_dir: Path to target directory.
    :param cycle: A datetime object to make available for use in the config.
    :param leadtime: A timedelta object to make available for use in the config.
    :param key_path: Path of keys to config block to use.
    :param dry_run: Do not create directories.
    :param stdin_ok: OK to read from ``stdin``?
    :return: A report on directories created / not created.



Directories to be created are specified by either a configuration YAML file or a Python ``dict``. A `makedirs` key must be included with a list of directories to create as its value.

In [27]:
%%bash
cat fixtures/fs/config/dir.yaml

makedirs:
  - foo
  - bar/baz


With these instructions, `makedirs()` creates each directory in the list within the directory indicated by `target_dir`. Paths can be provided either as a string or <a href="https://docs.python.org/3/library/pathlib.html#pathlib.Path">Path</a> object.  The return value of `makedirs()` is a `dict` reporting directories that were created (`ready`) and not created (`not-ready`).

In [28]:
rmtree("tmp/dir-target", ignore_errors=True)
fs.makedirs(
    config="fixtures/fs/config/dir.yaml",
    target_dir=Path("tmp/dir-target")
)

[2025-02-22T23:14:03]     INFO Validating config against internal schema: makedirs
[2025-02-22T23:14:03]     INFO 0 schema-validation errors found in fs config
[2025-02-22T23:14:03]     INFO Directory tmp/dir-target/foo: Executing
[2025-02-22T23:14:03]     INFO Directory tmp/dir-target/foo: Ready
[2025-02-22T23:14:03]     INFO Directory tmp/dir-target/bar/baz: Executing
[2025-02-22T23:14:03]     INFO Directory tmp/dir-target/bar/baz: Ready
[2025-02-22T23:14:03]     INFO Directories: Ready


{'ready': ['tmp/dir-target/foo', 'tmp/dir-target/bar/baz'], 'not-ready': []}

Examining the target directory, we can see that the directories have been created with their specified names.

In [29]:
%%bash
tree tmp/dir-target

tmp/dir-target
├── bar
│   └── baz
└── foo

4 directories, 0 files


### Using the `key_path` Argument  <!--dir-->

Consider the following configuration, in which the destination/source mapping is not located at the top level of the configuration:

In [30]:
%%bash
cat fixtures/fs/config/dir-keys.yaml

path:
  to:
    dirs:
      makedirs:
        - foo/bar
        - baz


Without additional information, `makedirs()` would raise a `UWConfigError` given this configuration. However, the list of keys leading to the destination/source mapping can be provided with the `key_path` argument:

In [31]:
rmtree("tmp/dir-keys-target", ignore_errors=True)
fs.makedirs(
    config="fixtures/fs/config/dir-keys.yaml",
    target_dir="tmp/dir-keys-target",
    key_path=["path","to","dirs"]
)

[2025-02-22T23:14:03]     INFO Validating config against internal schema: makedirs
[2025-02-22T23:14:03]     INFO 0 schema-validation errors found in fs config
[2025-02-22T23:14:03]     INFO Directory tmp/dir-keys-target/foo/bar: Executing
[2025-02-22T23:14:03]     INFO Directory tmp/dir-keys-target/foo/bar: Ready
[2025-02-22T23:14:03]     INFO Directory tmp/dir-keys-target/baz: Executing
[2025-02-22T23:14:03]     INFO Directory tmp/dir-keys-target/baz: Ready
[2025-02-22T23:14:03]     INFO Directories: Ready


{'ready': ['tmp/dir-keys-target/foo/bar', 'tmp/dir-keys-target/baz'],
 'not-ready': []}

With this information provided, the directories are successfully created.

In [32]:
%%bash
tree tmp/dir-keys-target

tmp/dir-keys-target
├── baz
└── foo
    └── bar

4 directories, 0 files


### Using the `MakeDirs` Class

An alternative to using `makedirs()` is to instantiate a `MakeDirs` object , then call its `go()` method.

In [33]:
help(fs.MakeDirs)

Help on class MakeDirs in module uwtools.fs:

class MakeDirs(Stager)
 |  MakeDirs(config: Union[dict, str, pathlib.Path, NoneType] = None, target_dir: Union[str, pathlib.Path, NoneType] = None, cycle: Optional[datetime.datetime] = None, leadtime: Optional[datetime.timedelta] = None, key_path: Optional[list[Union[bool, float, int, str]]] = None) -> None
 |  
 |  Make directories.
 |  
 |  Method resolution order:
 |      MakeDirs
 |      Stager
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  go(self)
 |      Make directories.
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __abstractmethods__ = frozenset()
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Stager:
 |  
 |  __init__(self, config: Union[dict, str, pathlib.Path, NoneType] = None, target_dir: Union[str, pathlib.Path, NoneType] = None, cycle: Optional[dat

A `MakeDirs` object is instantiated using the same parameters as `makedirs()`, but directories are not created until `MakeDirs.go()` is called.

In [34]:
rmtree("tmp/makedirs-target", ignore_errors=True)
dirs_stager = fs.MakeDirs(
    config="fixtures/fs/config/dir.yaml",
    target_dir="tmp/makedirs-target"
)
dirs_stager.go()

[2025-02-22T23:14:03]     INFO Validating config against internal schema: makedirs
[2025-02-22T23:14:03]     INFO 0 schema-validation errors found in fs config
[2025-02-22T23:14:03]     INFO Directory tmp/makedirs-target/foo: Executing
[2025-02-22T23:14:03]     INFO Directory tmp/makedirs-target/foo: Ready
[2025-02-22T23:14:03]     INFO Directory tmp/makedirs-target/bar/baz: Executing
[2025-02-22T23:14:03]     INFO Directory tmp/makedirs-target/bar/baz: Ready
[2025-02-22T23:14:03]     INFO Directories: Ready


Directories <281472332268544>

Once `MakeDirs.go()` is called, directories are created in the same way as they would have with `makedirs()`.

In [35]:
%%bash
tree tmp/makedirs-target

tmp/makedirs-target
├── bar
│   └── baz
└── foo

4 directories, 0 files


## Using glob Patterns

Python [glob patterns](https://docs.python.org/3/library/glob.html) can be used to copy multiple files with a single line of configuration. For example, consider this source directory:

In [36]:
%%bash
tree fixtures/fs/data

fixtures/fs/data
├── file1.nml
├── file2.txt
├── file3.csv
├── subdir1
│   └── file4.nml
└── subdir2
    └── file5.nml

3 directories, 5 files


All the `file*.*` files from the _top-level directory_ can be copied with this config:

In [37]:
%%bash
cat fixtures/fs/config/glob-copy.yaml

glob-copy/<f>: !glob fixtures/fs/data/file*.*


In [38]:
rmtree("tmp/glob-copy", ignore_errors=True)
fs.copy(
    config="fixtures/fs/config/glob-copy.yaml",
    target_dir="tmp",
)

[2025-02-22T23:14:03]     INFO Validating config against internal schema: files-to-stage
[2025-02-22T23:14:03]     INFO 0 schema-validation errors found in fs config
[2025-02-22T23:14:03]     INFO File fixtures/fs/data/file3.csv: Ready
[2025-02-22T23:14:03]     INFO File fixtures/fs/data/file1.nml: Ready
[2025-02-22T23:14:03]     INFO File fixtures/fs/data/file2.txt: Ready
[2025-02-22T23:14:03]     INFO Copy fixtures/fs/data/file3.csv -> tmp/glob-copy/file3.csv: Executing
[2025-02-22T23:14:03]     INFO Copy fixtures/fs/data/file3.csv -> tmp/glob-copy/file3.csv: Ready
[2025-02-22T23:14:03]     INFO Copy fixtures/fs/data/file1.nml -> tmp/glob-copy/file1.nml: Executing
[2025-02-22T23:14:03]     INFO Copy fixtures/fs/data/file1.nml -> tmp/glob-copy/file1.nml: Ready
[2025-02-22T23:14:03]     INFO Copy fixtures/fs/data/file2.txt -> tmp/glob-copy/file2.txt: Executing
[2025-02-22T23:14:03]     INFO Copy fixtures/fs/data/file2.txt -> tmp/glob-copy/file2.txt: Ready
[2025-02-22T23:14:03]     INFO

{'ready': ['tmp/glob-copy/file3.csv',
  'tmp/glob-copy/file1.nml',
  'tmp/glob-copy/file2.txt'],
 'not-ready': []}

The target directory after copying:

In [39]:
%%bash
tree tmp/glob-copy

tmp/glob-copy
├── file1.nml
├── file2.txt
└── file3.csv

1 directory, 3 files


The rightmost component in the destination-path key, `<f>`, is a placeholder, to be replaced with each source file matching the glob pattern. The `!glob` custom YAML tag instructs `fs.copy()` to treat the source-path value as a [glob pattern](https://docs.python.org/3/library/glob.html).

Recursive copies are supported via the `**` glob pattern:

In [40]:
%%bash
cat fixtures/fs/config/glob-copy-recursive.yaml

glob-copy-recursive/<f>: !glob fixtures/fs/data/**/*.nml


In [41]:
rmtree("tmp/glob-copy-recursive", ignore_errors=True)
fs.copy(
    config="fixtures/fs/config/glob-copy-recursive.yaml",
    target_dir=Path("tmp")
)

[2025-02-22T23:14:03]     INFO Validating config against internal schema: files-to-stage
[2025-02-22T23:14:03]     INFO 0 schema-validation errors found in fs config
[2025-02-22T23:14:03]     INFO File fixtures/fs/data/file1.nml: Ready
[2025-02-22T23:14:03]     INFO File fixtures/fs/data/subdir1/file4.nml: Ready
[2025-02-22T23:14:03]     INFO File fixtures/fs/data/subdir2/file5.nml: Ready
[2025-02-22T23:14:03]     INFO Copy fixtures/fs/data/file1.nml -> tmp/glob-copy-recursive/file1.nml: Executing
[2025-02-22T23:14:03]     INFO Copy fixtures/fs/data/file1.nml -> tmp/glob-copy-recursive/file1.nml: Ready
[2025-02-22T23:14:03]     INFO Copy fixtures/fs/data/subdir1/file4.nml -> tmp/glob-copy-recursive/subdir1/file4.nml: Executing
[2025-02-22T23:14:03]     INFO Copy fixtures/fs/data/subdir1/file4.nml -> tmp/glob-copy-recursive/subdir1/file4.nml: Ready
[2025-02-22T23:14:03]     INFO Copy fixtures/fs/data/subdir2/file5.nml -> tmp/glob-copy-recursive/subdir2/file5.nml: Executing
[2025-02-22T2

{'ready': ['tmp/glob-copy-recursive/file1.nml',
  'tmp/glob-copy-recursive/subdir1/file4.nml',
  'tmp/glob-copy-recursive/subdir2/file5.nml'],
 'not-ready': []}

The target directory after copying:

In [42]:
%%bash
tree tmp/glob-copy-recursive

tmp/glob-copy-recursive
├── file1.nml
├── subdir1
│   └── file4.nml
└── subdir2
    └── file5.nml

3 directories, 3 files


Note that `fs.copy()` ignores directories:

In [43]:
%%bash
cat fixtures/fs/config/glob-copy-ignore-dirs.yaml

glob-copy-ignore-dirs/<f>: !glob fixtures/fs/data/subdir*


In [44]:
fs.copy(
    config="fixtures/fs/config/glob-copy-ignore-dirs.yaml",
    target_dir=Path("tmp")
)

[2025-02-22T23:14:03]     INFO Validating config against internal schema: files-to-stage
[2025-02-22T23:14:03]     INFO 0 schema-validation errors found in fs config
[2025-02-22T23:14:03]  WARNING Ignoring directory fixtures/fs/data/subdir1
[2025-02-22T23:14:03]  WARNING Ignoring directory fixtures/fs/data/subdir2
[2025-02-22T23:14:03]     INFO File copies: Ready


{'ready': [], 'not-ready': []}

Linking files is similar to copying:

In [45]:
%%bash
cat fixtures/fs/config/glob-link-recursive.yaml

glob-link-recursive/<f>: !glob fixtures/fs/data/**/*.nml


In [46]:
rmtree("tmp/glob-link-recursive", ignore_errors=True)
fs.link(
    config="fixtures/fs/config/glob-link-recursive.yaml",
    target_dir=Path("tmp")
)

[2025-02-22T23:14:03]     INFO Validating config against internal schema: files-to-stage
[2025-02-22T23:14:03]     INFO 0 schema-validation errors found in fs config
[2025-02-22T23:14:03]     INFO Filesystem item fixtures/fs/data/file1.nml: Ready
[2025-02-22T23:14:03]     INFO Filesystem item fixtures/fs/data/subdir1/file4.nml: Ready
[2025-02-22T23:14:03]     INFO Filesystem item fixtures/fs/data/subdir2/file5.nml: Ready
[2025-02-22T23:14:03]     INFO Link tmp/glob-link-recursive/file1.nml -> fixtures/fs/data/file1.nml: Executing
[2025-02-22T23:14:03]     INFO Link tmp/glob-link-recursive/file1.nml -> fixtures/fs/data/file1.nml: Ready
[2025-02-22T23:14:03]     INFO Link tmp/glob-link-recursive/subdir1/file4.nml -> fixtures/fs/data/subdir1/file4.nml: Executing
[2025-02-22T23:14:03]     INFO Link tmp/glob-link-recursive/subdir1/file4.nml -> fixtures/fs/data/subdir1/file4.nml: Ready
[2025-02-22T23:14:03]     INFO Link tmp/glob-link-recursive/subdir2/file5.nml -> fixtures/fs/data/subdir2/f

{'ready': ['tmp/glob-link-recursive/file1.nml',
  'tmp/glob-link-recursive/subdir1/file4.nml',
  'tmp/glob-link-recursive/subdir2/file5.nml'],
 'not-ready': []}

In [47]:
%%bash
tree tmp/glob-link-recursive

tmp/glob-link-recursive
├── file1.nml -> ../../fixtures/fs/data/file1.nml
├── subdir1
│   └── file4.nml -> ../../../fixtures/fs/data/subdir1/file4.nml
└── subdir2
    └── file5.nml -> ../../../fixtures/fs/data/subdir2/file5.nml

3 directories, 3 files


Note that, while `fs.copy()` ignores directories, `fs.link()` links them:

In [48]:
%%bash
cat fixtures/fs/config/glob-link-dirs.yaml

glob-link-dirs/<f>: !glob fixtures/fs/data/subdir*


In [49]:
rmtree("tmp/glob-link-dirs", ignore_errors=True)
fs.link(
    config="fixtures/fs/config/glob-link-dirs.yaml",
    target_dir=Path("tmp")
)

[2025-02-22T23:14:03]     INFO Validating config against internal schema: files-to-stage
[2025-02-22T23:14:03]     INFO 0 schema-validation errors found in fs config
[2025-02-22T23:14:03]     INFO Filesystem item fixtures/fs/data/subdir1: Ready
[2025-02-22T23:14:03]     INFO Filesystem item fixtures/fs/data/subdir2: Ready
[2025-02-22T23:14:03]     INFO Link tmp/glob-link-dirs/subdir1 -> fixtures/fs/data/subdir1: Executing
[2025-02-22T23:14:03]     INFO Link tmp/glob-link-dirs/subdir1 -> fixtures/fs/data/subdir1: Ready
[2025-02-22T23:14:03]     INFO Link tmp/glob-link-dirs/subdir2 -> fixtures/fs/data/subdir2: Executing
[2025-02-22T23:14:03]     INFO Link tmp/glob-link-dirs/subdir2 -> fixtures/fs/data/subdir2: Ready
[2025-02-22T23:14:03]     INFO File links: Ready


{'ready': ['tmp/glob-link-dirs/subdir1', 'tmp/glob-link-dirs/subdir2'],
 'not-ready': []}

In [50]:
%%bash
tree tmp/glob-link-dirs

tmp/glob-link-dirs
├── subdir1 -> ../../fixtures/fs/data/subdir1
└── subdir2 -> ../../fixtures/fs/data/subdir2

3 directories, 0 files


## Copying HTTP Sources

In [51]:
%%bash
cat fixtures/fs/config/copy-http.yaml

licenses/gpl: https://raw.githubusercontent.com/ufs-community/uwtools/refs/heads/main/LICENSE


In [52]:
rmtree("tmp/licenses", ignore_errors=True)
fs.copy(
    config="fixtures/fs/config/copy-http.yaml",
    target_dir=Path("tmp")
)

[2025-02-22T23:14:03]     INFO Validating config against internal schema: files-to-stage
[2025-02-22T23:14:03]     INFO 0 schema-validation errors found in fs config
[2025-02-22T23:14:04]     INFO Remote object https://raw.githubusercontent.com/ufs-community/uwtools/refs/heads/main/LICENSE: Ready
[2025-02-22T23:14:04]     INFO Copy https://raw.githubusercontent.com/ufs-community/uwtools/refs/heads/main/LICENSE -> tmp/licenses/gpl: Executing
[2025-02-22T23:14:04]     INFO Copy https://raw.githubusercontent.com/ufs-community/uwtools/refs/heads/main/LICENSE -> tmp/licenses/gpl: Ready
[2025-02-22T23:14:04]     INFO File copies: Ready


{'ready': ['tmp/licenses/gpl'], 'not-ready': []}

In [53]:
%%bash
tree tmp/licenses

tmp/licenses
└── gpl

1 directory, 1 file
